# Load Model

In [2]:
from model import Mamba, ModelArgs
from transformers import AutoTokenizer

# One of:
#     'state-spaces/mamba-2.8b-slimpj'
#     'state-spaces/mamba-2.8b'
#     'state-spaces/mamba-1.4b'
#     'state-spaces/mamba-790m'
#     'state-spaces/mamba-370m'
#     'state-spaces/mamba-130m'
pretrained_model_name = 'state-spaces/mamba-790m'

model = Mamba.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

c:\Users\New-Hitesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Generate Text

In [3]:
import torch
import torch.nn.functional as F


def generate(model,
             tokenizer,
             prompt: str,
             n_tokens_to_gen: int = 50,
             sample: bool = True,
             top_k: int = 40):
    model.eval()
    
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    
    for token_n in range(n_tokens_to_gen):
        with torch.no_grad():
            indices_to_input = input_ids
            next_token_logits = model(indices_to_input)[:, -1]
        
        probs = F.softmax(next_token_logits, dim=-1)
        (batch, vocab_size) = probs.shape
        
        if top_k is not None:
            (values, indices) = torch.topk(probs, k=top_k)
            probs[probs < values[:, -1, None]] = 0
            probs = probs / probs.sum(axis=1, keepdims=True)
        
        if sample:
            next_indices = torch.multinomial(probs, num_samples=1)
        else:
            next_indices = torch.argmax(probs, dim=-1)[:, None]
        
        input_ids = torch.cat([input_ids, next_indices], dim=1)

    output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]
    
    return output_completions

# 130 Million Params

In [6]:
print(generate(model, tokenizer, 'Mamba is the')) #130m

Mamba is the most successful Mamba that has ever entered the game. After his amazing win over the Brazilian Mamba in Mombasa, he had only two more goals to his name the following week.

Mamba got his first call up to the M


In [7]:
print(generate(model, tokenizer, 'John: Hi!\nSally:')) #130m

John: Hi!
Sally: Hey!
John: This is Sally! (He's smiling a little at the show!)
(I also think we made up. Not sure!)
Sally: Hey there, John. I think it’s nice of us to introduce


In [10]:
print(generate(model, tokenizer, 'def reverse_string()')) #130m

def reverse_string() {
  return "Hello" :
}

And then my custom implementation of StringToHash:
private static StringToHash<String> getMessage(String message) {
  return (Message) MessageUtils.fromMessageBody


In [8]:
print(generate(model, tokenizer, 'The meaning of life is ')) #130m

The meaning of life is 
not defined 
by some simple definition 
or by some one-on-one interaction. 
If we are discussing philosophy, 
we are talking about the meaning of life. If we are discussing 
some other philosophical concept


# 790 Million Params

In [13]:
print(generate(model, tokenizer, 'The meaning of life is ')) #790m

The meaning of life is 
you need to realize that we are only here a few times. 
That's all. (laughs)
The world is here and gone.
You don’t need to know.

Russian: 
Поверь


In [15]:
print(generate(model, tokenizer, 'Hitesh is interning at ')) #790m

Hitesh is interning at ithink.io in New Delhi. Previously, he worked on Android for Google and iOS for Tata and NCR. He hails from Mumbai, is from an NGO and loves programming. He is an enthusiastic fan of both his dog and apple


In [4]:
print(generate(model, tokenizer, 'Hitesh is an intern. Elaborate on this. Invent necessary details')) #790m

Hitesh is an intern. Elaborate on this. Invent necessary details in front of the interviewer in front of you. Be bold with what you do.

Ask the interviewer a question he/she probably would have liked to have asked you. Be honest: the interviewer should be honest and expect questions


In [5]:
print(generate(model, tokenizer, 'A dog, a cat and a sheep walked into ')) #790m

A dog, a cat and a sheep walked into 
a barn.
Who would be the cat, the sheep or the dog?
The cat, because the cat is bigger than the sheep.
The dog, because the dog is better at hunting.
The sheep, because it lives in


In [6]:
print(generate(model, tokenizer, 'Tell me a joke ')) #790m

Tell me a joke.........................?"

"Oh I'm sorry....that's what you think? How many jokes I think that I can tell!.....I can't hear any one! I wish i could say 'I can tell any joke with any story
